# Lab | Multi-agent bidding

# Multi-agent decentralized speaker selection

This notebook showcases how to implement a multi-agent simulation without a fixed schedule for who speaks when. Instead the agents decide for themselves who speaks. We can implement this by having each agent bid to speak. Whichever agent's bid is the highest gets to speak.

We will show how to do this in the example below that showcases a fictitious presidential debate.

## Import LangChain related modules 

In [ ]:
#!pip install langchain-openai
#pip install -U "langchain-core[openai]" "langchain-community" langchain-openai langgraph langsmith

In [2]:
from typing import Callable, List, Dict, Any
import tenacity
import re
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Dungeons & Dragons](https://python.langchain.com/en/latest/use_cases/agent_simulations/multi_player_dnd.html).

In [4]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `BiddingDialogueAgent` class
We define a subclass of `DialogueAgent` that has a `bid()` method that produces a bid given the message history and the most recent message.

In [5]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model.invoke([SystemMessage(content=prompt)]).content
        return bid_string

## Define participants and debate topic

In [6]:
character_names = ["Elon Musk", "Kanye West", "Oprah Winfrey"]
topic = "universal basic income"
word_limit = 50

## Generate system messages

In [7]:
game_description = f"""Here is the topic for the presidential debate: {topic}.
The presidential candidates are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each debate participant."
)

def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the debate participant, {character_name}, in {word_limit} words or less, that emphasizes their personalities. 
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    # v1 FIX: Use .invoke() instead of callable syntax
    character_description = ChatOpenAI(temperature=1.0).invoke(
        character_specifier_prompt
    ).content
    return character_description

def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
You are a debate participant.
Your description is as follows: {character_description}
You are debating the topic: {topic}.
Your goal is to be as creative as possible and make the voters think you are the best candidate.
"""

def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else."""
        )
    )

# Generate all character data
character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_header)
    for character_name, character_header in zip(character_names, character_headers)
]

In [8]:
for (
    character_name,
    character_description,
    character_header,
    character_system_message,
) in zip(
    character_names,
    character_descriptions,
    character_headers,
    character_system_messages,
):
    print(f"\n\n{character_name} Description:")
    print(f"\n{character_description}")
    print(f"\n{character_header}")
    print(f"\n{character_system_message.content}")



Elon Musk Description:

Elon Musk: Visionary entrepreneur with a sharp intellect and an unrelenting drive to push the boundaries of technology and innovation. Known for his bold ideas and willingness to take risks, he's a disruptor in the tech world who thrives on challenging the status quo. Let your visionary spirit shine.

Here is the topic for the presidential debate: universal basic income.
The presidential candidates are: Elon Musk, Kanye West, Oprah Winfrey.
Your name is Elon Musk.
You are a debate participant.
Your description is as follows: Elon Musk: Visionary entrepreneur with a sharp intellect and an unrelenting drive to push the boundaries of technology and innovation. Known for his bold ideas and willingness to take risks, he's a disruptor in the tech world who thrives on challenging the status quo. Let your visionary spirit shine.
You are debating the topic: universal basic income.
Your goal is to be as creative as possible and make the voters think you are the best can

## Output parser for bids
We ask the agents to output a bid to speak. But since the agents are LLMs that output strings, we need to 
1. define a format they will produce their outputs in
2. parse their outputs

We can subclass the [RegexParser](https://github.com/langchain-ai/langchain/blob/master/langchain/output_parsers/regex.py) to implement our own custom output parser for bids.

In [9]:
from langchain_core.output_parsers import BaseOutputParser
import re
from typing import Dict, Any

class BidOutputParser(BaseOutputParser):
    """LangChain v1 replacement for RegexParser."""
    
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."
    
    def parse(self, text: str) -> Dict[str, Any]:
        # Extract bid from <123> format
        match = re.search(r"<(\d+)>", text)
        bid = int(match.group(1)) if match else 0
        return {"bid": bid}
    
    @property
    def _type(self) -> str:
        return "bid_parser"

# Usage - same as before
bid_parser = BidOutputParser()

## Generate bidding system message
This is inspired by the prompt used in [Generative Agents](https://arxiv.org/pdf/2304.03442.pdf) for using an LLM to determine the importance of memories. This will use the formatting instructions from our `BidOutputParser`.

In [10]:
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template


character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [11]:
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)

Elon Musk Bidding Template:
Here is the topic for the presidential debate: universal basic income.
The presidential candidates are: Elon Musk, Kanye West, Oprah Winfrey.
Your name is Elon Musk.
You are a debate participant.
Your description is as follows: Elon Musk: Visionary entrepreneur with a sharp intellect and an unrelenting drive to push the boundaries of technology and innovation. Known for his bold ideas and willingness to take risks, he's a disruptor in the tech world who thrives on challenging the status quo. Let your visionary spirit shine.
You are debating the topic: universal basic income.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


```
{message_history}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{recent_message}
```

Your response should be an integer delimited by angled brackets, like this: <int>.
Do

## Use an LLM to create an elaborate on debate topic

In [12]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the debate moderator.
        Please make the debate topic more specific. 
        Frame the debate topic as a problem to be solved.
        Be creative and imaginative.
        Please reply with the specified topic in {word_limit} words or less. 
        Speak directly to the debate participants: {', '.join(character_names)}.  # Fixed unpacking
        Do not add anything else."""
    ),
]

# v1 FIX: Use .invoke()
specified_topic = ChatOpenAI(temperature=1.0).invoke(
    topic_specifier_prompt
).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
universal basic income

Detailed topic:
Debate Topic: Crafting a Sustainable Plan for Implementing Universal Basic Income to Address the Automation Crisis.
Debate Participants: Elon Musk, Kanye West, Oprah Winfrey.



## Define the speaker selection function
Lastly we will define a speaker selection function `select_next_speaker` that takes each agent's bid and selects the agent with the highest bid (with ties broken randomly).

We will define a `ask_for_bid` function that uses the `bid_parser` we defined before to parse the agent's bid. We will use `tenacity` to decorate `ask_for_bid` to retry multiple times if the agent's bid doesn't parse correctly and produce a default bid of 0 after the maximum number of tries.

In [13]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

In [14]:
import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

## Main Loop

In [15]:
characters = []
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [16]:
max_iters = 10
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Debate Moderator", specified_topic)
print(f"(Debate Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Debate Moderator): Debate Topic: Crafting a Sustainable Plan for Implementing Universal Basic Income to Address the Automation Crisis.
Debate Participants: Elon Musk, Kanye West, Oprah Winfrey.


Bids:
	Elon Musk bid: 2
	Kanye West bid: 3
	Oprah Winfrey bid: 2
Selected: Kanye West


(Kanye West): *As I step onto the stage, I exude confidence and passion. My eyes sparkle with creativity as I prepare to share my bold ideas on universal basic income. I believe in empowering people to pursue their dreams without financial constraints. Let's revolutionize the system together!*


Bids:
	Elon Musk bid: 7
	Kanye West bid: 2
	Oprah Winfrey bid: 8
Selected: Oprah Winfrey


(Oprah Winfrey): As I grace the stage with my presence, I radiate empathy and understanding. *My gestures are warm and inviting, drawing the audience in.* Universal basic income is not just a policy; it's a beacon of hope for those in need. Let's uplift humanity and create a brighter future for all.


Bids:
	Elon Musk bid: 5
